# <span style='color:red'> Electric Aircraft Design </span>
&nbsp;    
A tool for scaling analysis of aircraft from prototype to full-scale aircraft
&nbsp;  
&nbsp;  
&nbsp;  

Prepared by William Schram  
  
B.S. Mechanical Engineering, Cal Poly SLO  
M.S. Management Science and Engineering, Stanford  
Commercial Instrument Pilot, CFI, CFII  
  
Last Revision: October 16, 2023  

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Problem" data-toc-modified-id="Problem-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problem</a></span></li><li><span><a href="#Given" data-toc-modified-id="Given-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Given</a></span></li><li><span><a href="#Assumptions" data-toc-modified-id="Assumptions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Assumptions</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analysis</a></span><ul class="toc-item"><li><span><a href="#Weight-of-Vehicle" data-toc-modified-id="Weight-of-Vehicle-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Weight of Vehicle</a></span></li><li><span><a href="#Airfoil-Selection" data-toc-modified-id="Airfoil-Selection-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Airfoil Selection</a></span></li><li><span><a href="#Power-from-Velocity-Requirement" data-toc-modified-id="Power-from-Velocity-Requirement-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Power from Velocity Requirement</a></span></li><li><span><a href="#Electric-Battery-Pack-Specifications" data-toc-modified-id="Electric-Battery-Pack-Specifications-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Electric Battery Pack Specifications</a></span></li></ul></li><li><span><a href="#Aircraft-Performance" data-toc-modified-id="Aircraft-Performance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Aircraft Performance</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Additional-Thoughts" data-toc-modified-id="Additional-Thoughts-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Additional Thoughts</a></span></li><li><span><a href="#Appendix:-Hobby-Motor-Specifications" data-toc-modified-id="Appendix:-Hobby-Motor-Specifications-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Appendix: Hobby Motor Specifications</a></span></li><li><span><a href="#Brushless-Motor-Design-Tools" data-toc-modified-id="Brushless-Motor-Design-Tools-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Brushless Motor Design Tools</a></span><ul class="toc-item"><li><span><a href="#Conductor-Lenght-per-Phase" data-toc-modified-id="Conductor-Lenght-per-Phase-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Conductor Lenght per Phase</a></span></li><li><span><a href="#Brushless-Motor-Parameterization" data-toc-modified-id="Brushless-Motor-Parameterization-10.2"><span class="toc-item-num">10.2&nbsp;&nbsp;</span>Brushless Motor Parameterization</a></span></li></ul></li></ul></div>

## Import Libraries

In [1]:
from math import pi as pi
from math import atan as atan
from math import sin as sin
from math import cos as cos
from math import log10 as log10
from math import degrees as degrees
from math import sqrt as sqrt

## Problem

Determine Runtime and Range for an Electric Drone Aircraft, given Weight, Speed, and Battery 
Capacity Parameters.


<b>Vehicle Specifications:</b>  
>Wing span : 2122mm  
Wing area : 80dm2  
Valid payload : 1000-2000g  

<b>Required for flying :</b>  
>Propeller: 12x6 or 12x8 (Foldable);  
Servos: 2 * 20g (the size of servo hole on the plane is 40*40mm);  
Controlller: 60A ESC or bigger;  
Capacity: 4s 6000---10000 mah li-Ion 

<b>Ideal Capabilities:</b>  
>60 knots cruise speed  
\> 40 mi range  
\> 30 min Flight Duration 

<b>Future Upgrades:</b>  
>0) Stall speed  
1) Renolds number: Laminar vs. Turbulent transition  
2) Verify atan() is in correct units!  

## Given

In [7]:
## Air Properties:
T_0 = 15 # [C], Temperature 12-15*C
P_0 = 101.325 # [N/m^2], Pressure (Standard)
R_0 = 0.3 # [-], Relative Humidity
RHO = 1.225 # [kg/m^3], Density # Pulled from EES
MU = 1.801e-5 # Dynamic Viscosity # Pulled from EES
NU = MU/RHO # [m^2/s], Kinematic Viscosity: ~1.477E-5 [m^2/s]

## Vehicle / Wing Parameters:
A_p = 0.8 # [m^2], Area Projected
b = 2.122 # [m], Wingspan Length (Projected)
c =  0.4 # [m], Chord Length (Projected, Nominal) # Width 'W'
# ar = 3.0 # [-], Aspect Ratio (Projected) # This Value is Calculated

## Weight of Vehicle:
g = 9.807 # [m/s^2], Acceleration from gravity
m_payload = 1.0 # [kg], 1-2 Kg payload capacity
m_aircraft = 3.5 # [kg]
m_batteries = 1 * 0.495 # [kg] # 0.980 kg = 980 gm
# m_cell = .245 # [kg], 980 gm / 4
m_motor = 0.132 # [kg]

## Speed in Flight
velocity_kts = 60 # [knots = km/hr]

## Assumptions

In [8]:
# 1) Rigid Body, Static vehicle dynamics traveling at v_bar
# 2) Turbulent Air Flow, Turbulent Flow == Re > 5x10^5
# 3) No Efficiency loss for the Motor / Controller
# 4) Discharge Safety factor of : 0.6
SF_Discharge = 0.65
# 5) Glide Ratio: i.e. 12:1  Source: https://en.wikipedia.org/wiki/Lift-to-drag_ratio
horizontal = 12.0; vertical = 1.0

## Analysis

### Weight of Vehicle

In [9]:
## Mass, Weight
m = m_payload + m_aircraft + m_batteries + m_motor; print('Mass =', round(m,2), '[Kg]') # [Kg]
W = m * g;
W_Eng = W * (1/4.4482) # [lb_f], (1 lb_f / 4.4482 N)
print('Weight =', round(W,2), '[N]','=',round(W_Eng,2),'[lb_f]') # [N]

Mass = 5.13 [Kg]
Weight = 50.28 [N] = 11.3 [lb_f]


### Airfoil Selection

Pg 376 - Fundamentals of Aerodynamics

<b>Airfoil Selection Literature:</b>  

<i>Fundamentals of Aerodynamics 6ed., 2017</i>  
Pg. 478-9: NACA 2412 Wing section

<i>NASA Low and Medium Speed Airfoils Development - NASA TM 78709 \[1980\]</i>
  
Low-speed airfoils:    
C_L = 0.4
Re = 2*10^6 - 9x10^6  
Thickness-chord ratio = .13 for C_L >2  

Medium-speed airfoils (Ex. Cessna 303, and Hustler - American Jet ):  
C_L = 0.3  
Re = 14*10^6  
Thickness-chord ratio = .13 and .17  
Mach = 0.72 and 0.68  

Airfoil Examples: NASA LS(1)-0417, NASA LS(1)-0409, NASA LS(1)-0413

In [10]:
## Assume a Airfoil -> C_L and C_D at Alpha, angle of attack, equals 0 [Rad].
# C_L = 0.25; C_D = 0.01 # NACA 2412
C_L = 0.4; C_D = .007 # NASA LS(1)-0413
# C_L = 0.44; C_D = 0.011 # NASA LS(1)-0417: 2.0E6 to 6.0E6

In [11]:
ar = b**2/A_p; print('Aspect Ratio =', round(ar,2), '[-]')
# taper_ratio = c_root / c_tip

Aspect Ratio = 5.63 [-]


### Power from Velocity Requirement

In [12]:
## V_cruise: Power to obtain cruise velocity  
# W = 1/2 * rho * velocity_ss ** 2 * A_p *C_L , Solve for Velocity  
# velocity_ss = sqrt(2 * W / (RHO * A_p * C_L)); # [m/s]  
velocity_ms = velocity_kts * (0.5144) # [kts]->[m/s]  
velocity_mph = velocity_kts * (1.1508); # [kts]->[mi/hr]  
print('Velocity =',round(velocity_kts,2),'[kts]','=',round(velocity_ms,2),'[m/s]','=',round(velocity_mph,2),'[mi/hr]')  

q = (1/2) * RHO * velocity_ms**2; print('Dynamic Pressure =', round(q,2),'[Kg/m^2]') # dynamic pressure  
F_Drag = q * A_p * C_D; print('Force of Drag =', round(F_Drag,2), '[N]') # [N], Force of Drag  

P_Shaft = F_Drag * velocity_ms; # [W]  
P_HP = P_Shaft * (1/745.7) # [W] to [Hp]  
print('Power =', round(P_Shaft,2),'[W or N-m/s]', '=', round(P_HP,4), '[Hp]')  

Velocity = 60 [kts] = 30.86 [m/s] = 69.05 [mi/hr]
Dynamic Pressure = 583.46 [Kg/m^2]
Force of Drag = 3.27 [N]
Power = 100.84 [W or N-m/s] = 0.1352 [Hp]


In [133]:
## C_d_min: Power to Remain Aloft at Economy Cruise (Less than required velocity)
# At level flight, Weight = Lift, therefore,
# W = 1/2 * rho * velocity_ss**2 * A_p *C_L , Solve for Velocity
velocity_ms = sqrt(2*W/(RHO * A_p * C_L)); # [m/s]
velocity_kts = velocity_ms *(1.9438/1);
velocity_mph = velocity_ms * (2.2369/1);
print('Velocity_stall =', round(velocity_kts,2),'[kts]','=', round(velocity_ms,2),'[m/s]','=', round(velocity_mph,2),'[mi/hr]')

q = (1/2) * RHO * velocity_ms**2; print('dynamic pressure =', round(q,2),'[Kg/m^2]') # Dynamic Pressure  
F_Drag = q * A_p * C_D; print('Force of Drag =', round(F_Drag,2), '[N]') # Force of Drag

P_Shaft = F_Drag * velocity_ms; 
P_HP = P_Shaft * (1/745.7) # Watts to Horsepower
print('Power =', round(P_Shaft,2),'[W or N-m/s]', '=', round(P_HP,4), '[Hp]')

Velocity_stall = 31.13 [kts] = 16.02 [m/s] = 35.83 [mi/hr]
dynamic pressure = 157.13 [Kg/m^2]
Force of Drag = 0.88 [N]
Power = 14.09 [W or N-m/s] = 0.0189 [Hp]


<b>Note:</b>  
• Hydrogen fuel cell and tank can deliver 800W for one hour  
• Lithium ion can deliver 500W for 20 minutes  
• Everthing else is piston or turbine powered!

### Electric Battery Pack Specifications

In [13]:
## Select Cell Chemistry
# -------- Li-ION Cells -------- (from Upgrade Energy)
V_Cell_Max = 4.2 # [v], Volts/Cell, Max Voltage
V_Cell_Min = 2.7 # Min Charge
Capacity_Cell = 1.05 # [A-h], Capacity/Cell
Discharge_Rate = 25 # [c], coulombs # I_cont. = A-h * C" "Guess"
I_cont = 45 # [A]
T_max = 140 # [°F]

# -------- LiPO Cells --------
# V_Cell_Max = 3.7 # [v], Volts/Cell, Max Voltage
# V_Cell_Min = 2.4 # Min Charge
# Capacity_Cell = 2.5 # [A-h], Capacity/Cell
# Discharge_Rate = 25 # [c], coulombs # I_cont. = A-h * C" "Guess"
# I_cont = 50 # [A]

In [14]:
## ------- Battery(s) --------  
s = 4 # Cells in Series  
p = 2 # Cells in Parallel  
numOfCells = s * p  

# 4s=14.8V, 6s=22.2V, 8s=29V 10S=37V, 12S=42V"  
V_Min = V_Cell_Min * s  
V_Max = V_Cell_Max * s # Max Charge (3.65 or 3.7???)  
V = (V_Max + V_Min)/2; print('Battery Voltage =', round(V,2),'[V]', '(',V_Max,'-',V_Min,'[V] )' )  

Capacity_Battery = Capacity_Cell * s * p; print('Capacity =', round(Capacity_Battery,2),'[A-h]')  
NRG_Battery = Capacity_Battery * V; print('Energy =', round(NRG_Battery,2), '[W-h]')  
I_Battery_Continuous = p * I_cont; print('I_Continuous =',round(I_Battery_Continuous,2), '[A]') # I = 90 [A]  

Battery Voltage = 13.8 [V] ( 16.8 - 10.8 [V] )
Capacity = 8.4 [A-h]
Energy = 115.92 [W-h]
I_Continuous = 90 [A]


In [15]:
## Supply Current
# 7 Amps for a flying wing, source: Randy Snowder, https://vimeo.com/41306784
I = P_Shaft / V; print('Current in Level Flight =', round(I,2), '[A]')

Current in Level Flight = 7.31 [A]


In [41]:
## ------- Brushless Motor Parameterization: -----------  
# See Hendershot and Miller for more information on motor constants.  
# To compare the following are needed:  kv [rpm/V], resistance [ohm], No-load Current [A],
#                                       Dimensions, Recommended Propeller Size

## T-motor for Hexsoon 450
K_v = 880 # [rpm/V], See note 2.
R_w = 0.115 # [Ohm] Alibaba 115+/-15mohms, Assume linetoline -> R_phase = 3/2 R_linetoline for Delta wound, 1/2 for wye wound
I_No_Load = 0.08 # [A/V], Alibaba: 0.6-0.8A at 10V
## Prop T-motor - T1045
# Optimal RPM 6000-7000 RPM

## Calcs
R_phase = 3/2 * R_w
I_0 = I_No_Load * V # I_0 = 2.08 # [A], No Load Current at rated voltage (10V Default)
Kq= 60/(2*pi) * sqrt(3/2) * K_v # [-], Torque Constant, second term accounts for note 2.

# Note(s):  
# 1. List of other motor specs is in the appendix.
# 2. K_v is line-to-line voltage not phase voltage, which underestimates delta wound torque by sqrt(2/3)
#    & overestimates wye wound torque by sqrt(2). https://youtu.be/jrWDBkeOVQY?si=rGsagyh8TB_5rONt&t=719

<img src="TMotor_AIR2216.png" width=800 height=400>

In [43]:
## Rotations and Torque at current I and voltage V  
N_Motor = K_v * (V - I * R_phase); print('N_motor =', int(N_Motor), '[rpm]') # N_motor = Kv * (Vin - VLoss), Ideally 7.2k-10k [rpm]  
omega = N_motor * (2*pi) / 60 # [rad/sec]  
# TAU_SI = Kq * (I - I_0) # [N-m], Torque = Kq * (I - Io) # If k_q is known!   
# TAU_Eng = TAU_SI * (0.7376/1); print('Torque =', round(TAU_Eng,2), '[ft-lb_f]') # (0.7376 lb_f / 1 N-m)"

## Motor Power, Efficiency and Heat at current (I)  
P_Motor = I * V; print('P_Motor =', round(P_Motor,2), '[W]')  
ETA_Motor = ( V*I - I**2*R_phase - V*I_0 ) / ( V*I ); print('Motor Efficiency =', round(ETA_Motor*100,1),'[%]') # "Sn-B"??
Q_dot_Motor = I**2 * R_phase #(1-ETA_Motor) * (I * V) # [W], Resistive heat generated by motor  

## Torque
TAU_SI = ETA_Motor * P_motor * 9.549 / N_Motor; # [N-m]  
TAU_Eng = TAU_SI * (0.7376/1); # (0.7376 lb_f / 1 N-m)"  
print('Torque =', round(TAU_SI,3), '[N-m] =', round(TAU_Eng,3), '[lb_f-ft] =', round(12*TAU_Eng,3), '[lb_f-in]');  

Runtime = NRG_Battery / P_Motor # [h], Hours"  
Runtime_min = NRG_Battery / P_Motor * (60/1) # [min], (60min/1h)  
print('Battery Runtime =', round(Runtime,2), '[hr] =', round(Runtime_min,2), '[min]')  

N_motor = 11034 [rpm]
P_Motor = 100.84 [W]
Motor Efficiency = 75.8 [%]
Torque = 0.066 [N-m] = 0.049 [lb_f-ft] = 0.585 [lb_f-in]
Battery Runtime = 1.15 [hr] = 68.97 [min]


In [4]:
##  Inductance of Brushless AC Motor (Ns=10,Nr=12)
num_poles = 10 # [-]
mew = 1.25663753 * 10**(-6) # permeability of air = 1.25663753×10−6
N = 4 # Number of turns per pole
A = .023**2 / 2 # [m^2]
l = 4.2886 # [m] # lenght of the wire
L_pole = mew * N**2 * A / l # Inductance of a pole
L = num_poles * L_pole; L # [H] 

1.2400527041365477e-08

## Aircraft Performance

In [17]:
## ------- Wing Parameters: Lift, Drag, and Thrust Forces...: ---------
Alpha = atan(vertical/horizontal) # [Rad]
print('Alpha = ', round(degrees(Alpha),2),'Degrees') # [Degrees]

ar_calc = b**2/(A_p); print('AR = ', round(ar_calc,2),'[-]')

Alpha =  4.76 Degrees
AR =  5.63 [-]


In [20]:
## ------- Summation of forces in X & Y --------
# Equ 1: Sum(F_y) = 0
#        F_Lift*cos(Alpha) - m*g = 0 # [N]
F_Lift = m*g / cos(Alpha); print('F_Lift =', round(F_Lift,2), '[N]')

# Equ 2: Sum(F_x) = 0
#        F_Lift*sin(Alpha) - F_Drag = 0 # [N]
F_Drag = F_Lift*sin(Alpha); print('F_Drag =', round(F_Drag,2), '[N]')

# Equ 3: F_Lift = 1/2 * RHO * v_ss^2 * A_p * C_L # [N]
#        C_L = F_Lift * 2 / (RHO * v_ss^2 * A_p) 
C_L_calc = F_Lift * 2 / (RHO * velocity_ms**2 * A_p); print('C_L_calc =', round(C_L_calc,4), '[-]') # Coefficient of Lift

# Reynolds Number
Re_L = (RHO * velocity_ms * c) / MU; print('Reynold\'s Number =', f"{Re_L:.2E}", '[-]') # [-], OR v * c / NU

F_Lift = 50.45 [N]
F_Drag = 4.19 [N]
C_L_calc = 0.1081 [-]
Reynold's Number = 8.40E+05 [-]


In [21]:
# Force of Drag, F_D
C_D_calc = 0.455 / (log10(Re_L) )**(2.58) - 1610 / (Re_L); print('C_D_calc =', round(C_D,4), '[-]') # [Coeficient of Drag] 
# F_Drag = C_D_calc * 1/2 * RHO * v_ss**2 * A_p; print('F_Drag = ', round(F_Drag,2),'[N]') # [N]

# CoefLiftToDragRatio = C_L_calc / C_D_calc;  print('Coef. Lift/Drag = ', int(CoefLiftToDragRatio),'[-]') # [-]
LiftToDragRatio = F_Lift / F_Drag;  print('Lift/Drag Ratio = ', int(LiftToDragRatio),'[-]') # [-]

C_D_calc = 0.007 [-]
Lift/Drag Ratio =  12 [-]


In [23]:
## ------- Convertion Metric to English: --------  
velocity_ss_mph = velocity_ms * (2.23694/1); print('Velocity_ss = ', round(velocity_mph,2),'[mi/hr]') # m/s -> mi/hr
velocity_x_mph = cos(Alpha) * velocity_mph; print('Horizontal Velocity_ss = ', round(velocity_x_mph,2),'[mi/hr]') # [mi/hr]
velocity_y_mph = sin(Alpha) * velocity_mph; print('Vertical Velocity_ss = ', round(velocity_y_mph,2),'[mi/hr]') # [mi/hr]

Velocity_ss =  69.05 [mi/hr]
Horizontal Velocity_ss =  68.81 [mi/hr]
Vertical Velocity_ss =  5.73 [mi/hr]


In [24]:
## ------- Range -------- 
Range = Runtime * velocity_ss_mph; print('Range = ', round(Range,2),'mi') # [mi]

Range =  79.36 mi


In [27]:
## ------- Stopping Distance, per FAR 25.125 -------
# Values are in the landing configuration, assume no flaps / spoilers
# Then, 
v_50 = 1.3 * velocity_ms 
v_land = 1.25 * velocity_ms

## Distance to clear 50ft object and touchdown
## d_air = 50*(L/d_eff) + (v_50**2 - v_land**2)/(2a)
## d_air = 50*LiftToDragRatio + (velocity_ss**2 - v_land**2)/(2a)
## -OR- Assume: Lift = Weight
## d_air = L/d_eff*(50 + 1/(2*g)*(v_50**2-v_land**2) )
d_air = 50*LiftToDragRatio + LiftToDragRatio/(2*g)*(velocity_ms**2-v_land**2)
d_air_ft = d_air*(3.2808/1)
print('Landing Distance, per FAR 25.125')
print('Air Distance =', int(d_air),'[m]','or', int(d_air_ft),'[ft]')

## Ground deceleration distance
## d_gnd = V_L**2/(2*a) = V_L**2*W/(2*g*R), where R = mew*(W-L)+D and mew is coef of friction, D=spoiler/flaps drag
mew = 0.42 # Steel on steel
R = mew*(W-F_Lift) + F_Drag
d_gnd = 1.25*velocity_ms**2*W / (2*g*R)
d_gnd_ft = d_gnd*(3.2808/1)
print('Ground Distance =', int(d_gnd),'[m]','or', int(d_gnd_ft),'[ft]')

Landing Distance, per FAR 25.125
Air Distance = 273 [m] or 896 [ft]
Ground Distance = 741 [m] or 2432 [ft]


## Results

In [28]:
print('Current in Level Flight =', round(I,2), '[A]')
print('Battery Runtime =', round(Runtime,2), '[hr] =', round(Runtime_min), '[min]')  
print('Range = ', round(Range,2),'[mi]') # [mi]

Current in Level Flight = 7.31 [A]
Battery Runtime = 1.15 [hr] = 69 [min]
Range =  79.36 [mi]


## Additional Thoughts

In [119]:
from scipy.optimize import fsolve

import warnings
warnings.filterwarnings('ignore')

In [130]:
## Calculate Reynolds Number - Three range-based equations
def Re_L_Laminar_Calc(Re_L):
    '''    Reynolds Number, Laminar flow: Re_L < 5*10^5    '''
    global C_D
    return -C_D + 1.33/((Re_L)**(1/2))

def Re_L_Turbulent_Calc(Re_L):
    '''    Reynolds Number, Turbulent flow: 5*10^5 <= Re_L < 10^7    '''
    global C_D
    return -C_D + 0.0742/((Re_L)**(1/5))

def Re_L_Turbulent2_Calc(Re_L):
    '''    Reynolds Number, Turbulent flow: 10^7 <= Re_L < 10^9     '''
    global C_D
    return -C_D + 0.455/(log10(Re_L)**(2.58))

iter_solution = fsolve(Re_L_Laminar_Calc, 500000) # 
Re_L_Lam = iter_solution[0] # print(f"{Re_L_Lam:.2E}")

iter_solution = fsolve(Re_L_Turbulent_Calc, 500000) # 
Re_L_Turb = iter_solution[0] # print(f"{Re_L_Turb:.2E}") 

iter_solution = fsolve(Re_L_Turbulent2_Calc, 100000000) # 
Re_L_Turb2 = iter_solution[0] # print(f"{Re_L_Turb2:.2E}") 

if Re_L_Lam < 5*10**5:
    Re_L = Re_L_Lam
elif (Re_L_Turb >= 5*10**5) and (Re_L_Turb < 10**7):
    Re_L = Re_L_Turb
elif (Re_L_Turb >= 10**7) and (Re_L_Turb < 10**9):
    Re_L = Re_L_Turb2
else:
    print('Equations for Reynold\'s number > 10^9 needed!')

print('Reynold\'s Number =', f"{Re_L:.2E}", '[-]')

TypeError: fsolve() got an unexpected keyword argument 'Re_L0'

• Hydrogen Tank can deliver 800W for one hour


In [17]:
## Pseudo-code for docking mechanism
'''
Dock consists of four neodynium magnets horizontal (N) and verticle (S),
in a cross configuration
'''

# Test: Method and hardware must work against a pulsed lawn blower!

current_coord = (x,y,z,pitch,yaw,roll) # End of drone flight route
dock_coord, delta_coord = request_dock_coord(current_coord) # (x,y,z,pitch,yaw,roll), yaw is compass heading

start_dock_coord = berth_coord - (0,0,-10,0,0,0) # Third arg is offset in ft.
go_to(start_dock_coord)

def docking_pad_visible():
    # detect two markers in center of image signifying roll and distance is within 
    # Z distance (+/-10ft) and yaw range (+/-10°)
    return True, delta_coord

def docking_procedure(coord, delta_coord):
    velocity(0,0,+5,0,0,0)
    go_to_delta(delta_coord)
    
while docked()==False:
    request_dock_coordinate(current_coord)
    if docking_pad_visible(up_image)[0]==True) and (delta_coord[3]<0.5):
        docking_procedure(coord, docking_pad_visible(up_image)[1]) # Second arg is delta_coord
    elif delta_coord[3]<0.5:
        velocity(0,0,+1,0,0,0)
        sleep.sleep(2) # sleep 2 seconds allowing the craft to press into the dock 


SyntaxError: invalid syntax (<ipython-input-17-fff32a162236>, line 26)

In [ ]:
## Iteratively solve for battery current
from scipy.optimize import fsolve
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def current_in_disguise(I):
    '''
    P_battery = Q_dot_Motor + P_Shaft
    I * V = (V*I - I**2*R_w - V*I_0)/(V*I) + P_Shaft
    0 = -I*V + (V*I - I**2*R_w - V*I_0)/(V*I) + P_Shaft
    '''
    global V, R_w, P_Shaft
    return -I*V + (V*I - I**2*R_w - V*I_0) / (V*I) + P_Shaft

iter_solution = fsolve(current_in_disguise, P_Shaft/V) # Approximation is Power / Voltage, 
I = iter_solution[0]
print('Current =', round(I,2), '[A]')

## Appendix: Hobby Motor Specifications

In [ ]:
## SunnySky X3508S KV580 (Eff.=87%, price=$28)
# Kv = 580
# R_w = 0.135 # [Ohm]
# # I_No_Load = 0.11 # [A/V]  
# I_0 = 0.3 # I_No_Load * V # [A], No Load Current

## TMAT2820 - 7 (Eff.=68%)
# Kv = 830 # [rpm/V], TMAT2820-7: 830 rpm / Volt  
# R_w = 0.030 # [Ohm], 0.0132ohm @ Y 730kv / 0.0046 @ DeLta 1280kv  
# I_No_Load = 0.11 # [A/V]  
# I_0 = I_No_Load * V # No Load Current: 1.1A/10V @ Y 730kv / 4.2 @ DeLta 1280kv  

## Turnigy Multistar 3508-640Kv (Eff.=79%)
# Kv = 640 # [rpm/V]
# R_w = 0.12 # [Ohm]
# # I_No_Load = 0.11 # [A/V]  
# I_0 = 0.6 # I_No_Load * V # [A], No Load Current

## Scorpion SII-3020-1110 (Eff.=37%)
# Kv = 1110 # [rpm/V]
# R_w = 0.016 # [Ohm]
# # I_No_Load = 0.11 # [A/V]  
# I_0 = 2.08 # I_No_Load * V # [A]



## Brushless Motor Design Tools

### Conductor Lenght per Phase

In [2]:
## Right Triangle Coil
tail = 0.05 # 50mm Solder to flexible wires
arm = 0.023 # m, or 23 mm
coil_length = 2 * arm + (sqrt(2) * arm); # [m]
poles = 6 # [-]  
turns_per_pole = 6 # [-]  
winding_diameter = .051 # [m]
#      Total Length = Tails + Circle Length + Turns  
winding_length = (2 * tail) + (3.1415 * winding_diameter) + (poles * turns_per_pole * coil_length); 
print('Right Triangle Coil Length, per Phase =', round(winding_length,2), '[m]' )

Right Triangle Coil Length, per Phase = 3.09 [m]


### Brushless Motor Parameterization

In [54]:
## ------- Brushless Motor Parameterization: -----------  
# See Hendershot and Miller for more information on motor constants.  
# To compare the following are needed:  kv [rpm/V], resistance [ohm], No-load Current [A],
#                                       Dimensions, Recommended Propeller Size
# Note: K_v is line-to-line voltage not phase voltage, which underestimates delta wound torque by sqrt(2/3)
#    & overestimates wye wound torque by sqrt(2). https://youtu.be/jrWDBkeOVQY?si=rGsagyh8TB_5rONt&t=719

## Given:
I = 3 # [Amps]
V = 15.2 # [Volts], 4S == 15.2-16.8 Volts
winding = "wye" # "wye" or "delta"
pole_pairs = 6

## Measure:
# Measure at 10V for reporting hobby motors
K_v = 1675 # [rpm/V] # rpm per volt_line-to-line

# Measure average of three legs of the winding.
R_ll = 0.32 # [Ohm] # Line-to-line resistance, 

# Measure current draw at highest signal from ESC with no load.
I_No_Load = 0.11 # [A/V] # Guess. Use to calculate I_0.

# Drive Motor with drill and measure line-to-line voltage, cycles/sec
V_ll = 4.8 # [V], value will be V_AC_rms
omega = 2*pi * 151.5 # [rad/s] # Measure [Hertz], convert to [rad/s]: 151.5 [1/s]

## Calculations:  
I_0 = I_No_Load * V # [A], No Load Current at rated voltage (10V Default)

K_t = V_ll / (sqrt(2/3) * omega * (2*pi) / pole_pairs) # [N-m/A] or [V-s/rad], Torque Constant: Drive motor with drill and measure, See note 2. 

TAU_SI = 1/sqrt(2) * 1/K_v * I; # [N-m]
TAU_Eng = TAU_SI * (0.7376/1); # [ft-lb_f], (0.7376 ft-lb_f / 1 N-m)" 

# Power
P_in = I*V # [W]# Input Power

if winding == "delta": R_phase = 3/2 * R_ll # Compensates for delta or Wye connection.
elif winding == "wye": R_phase = 1/2 * R_ll
P_resistance = I**2 * R_phase; # [W], Copper power losses

P_other = I_0 * R_phase**2 # [W], Iron and bearing losses

P_out = P_in - P_resistance - P_other

# Efficiency
eta = P_out / P_in

# Reporting:
print('=== Summary ===')
print('    ')
print('K_v = ', K_v, '[RPM/V]')
print('K_t =', round(K_t,2)*1000, '[mN-m/A]' ) # Hobby motors described in mN-m/A .
print('Torque =', round(Torque,4), "[N-m] =", round(TAU_Eng,4), "[ft-lb_f]" )# Torque # [N-m]
print('    ')
print("Power In =", round(P_in,2) , '[W]') 
print('Power Loss (Resistive)  =', P_resistance,"[W]"  )
print('Power Loss (Iron/Bearings) =', round(P_other,2), '[W]')
print('Power Out =', round(P_out), '[W]')
print('Efficiency =', round(eta*100,1), '[%]')

=== Summary ===
    
K_v =  1675 [RPM/V]
K_t = 10.0 [mN-m/A]
Torque = 0.0042 [N-m] = 0.0009 [ft-lb_f]
    
Power In = 45.6 [W]
Power Loss (Resistive)  = 1.44 [W]
Power Loss (Iron/Bearings) = 0.04 [W]
Power Out = 44 [W]
Efficiency = 96.7 [%]
